In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)

/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sunxin/minico

1.13.2
2.2.4


Using TensorFlow backend.


In [2]:
import keras
import numpy as np
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers

In [3]:
batch_size = 32
num_classes = 10
# epochs = 100
epochs = 50
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'models')
model_name = 'keras_cifar10_trained_model.h5'

In [4]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), padding='same',
                        input_shape=x_train.shape[1:]))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(32, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, (3, 3), padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(64, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(512))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))
model.summary()

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)  

In [6]:
print('Not using data augmentation.')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

Not using data augmentation.
Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 14s 281us/step - loss: 1.8505 - acc: 0.3209 - val_loss: 1.5592 - val_acc: 0.4333
Epoch 2/50
50000/50000 [==============================] - 12s 239us/step - loss: 1.5012 - acc: 0.4585 - val_loss: 1.3671 - val_acc: 0.5111
Epoch 3/50
50000/50000 [==============================] - 12s 239us/step - loss: 1.3671 - acc: 0.5087 - val_loss: 1.3303 - val_acc: 0.5336
Epoch 4/50
50000/50000 [==============================] - 12s 241us/step - loss: 1.2675 - acc: 0.5509 - val_loss: 1.2414 - val_acc: 0.5690
Epoch 5/50
50000/50000 [==============================] - 12s 238us/step - loss: 1.1888 - acc: 0.5807 - val_loss: 1.0804 - val_acc: 0.6182
Epoch 6/50
50000/50000 [==============================] - 11s 227us/step - loss: 1.1193 - acc: 0.6057 - val_loss: 1.0482 - val_acc: 0.6297
Epoch 7/50
50000/50000 [===============

In [7]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at /home/sunxin/qhm_experiment/experiments/models/keras_cifar10_trained_model.h5 
10000/10000 [==============================] - 1s 67us/step
Test loss: 0.6546306056022644
Test accuracy: 0.7854


In [8]:
print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.,  # set range for random shear
    zoom_range=0.,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    epochs=epochs,
                    steps_per_epoch=int(np.ceil(x_train.shape[0] / batch_size)),
                    validation_data=(x_test, y_test),
                    workers=4)

Using real-time data augmentation.
Epoch 1/50
1563/1563 [==============================] - 23s 15ms/step - loss: 0.8644 - acc: 0.7128 - val_loss: 0.8001 - val_acc: 0.7364
Epoch 2/50
1563/1563 [==============================] - 23s 15ms/step - loss: 0.8550 - acc: 0.7141 - val_loss: 0.7052 - val_acc: 0.7665
Epoch 3/50
1563/1563 [==============================] - 23s 15ms/step - loss: 0.8529 - acc: 0.7134 - val_loss: 0.8309 - val_acc: 0.7294
Epoch 4/50
1563/1563 [==============================] - 23s 15ms/step - loss: 0.8417 - acc: 0.7186 - val_loss: 0.8332 - val_acc: 0.7442
Epoch 5/50
1563/1563 [==============================] - 23s 15ms/step - loss: 0.8363 - acc: 0.7202 - val_loss: 0.7715 - val_acc: 0.7486
Epoch 6/50
1563/1563 [==============================] - 23s 15ms/step - loss: 0.8363 - acc: 0.7201 - val_loss: 0.8178 - val_acc: 0.7468
Epoch 7/50
1563/1563 [==============================] - 23s 15ms/step - loss: 0.8348 - acc: 0.7203 - val_loss: 0.7722 - val_acc: 0.7460
Epoch 8/50
15

In [9]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at /home/sunxin/qhm_experiment/experiments/models/keras_cifar10_trained_model.h5 
10000/10000 [==============================] - 1s 77us/step
Test loss: 0.843899684047699
Test accuracy: 0.7345
